# Тема “Работа с данными в Pandas”

### Задание 1
Импортируйте библиотеку Pandas и дайте ей псевдоним pd. Создайте датафрейм authors со столбцами author_id и author_name, в которых соответственно содержатся данные: [1, 2, 3] и ['Тургенев', 'Чехов', 'Островский'].
Затем создайте датафрейм book cо столбцами author_id, book_title и price, в которых соответственно содержатся данные: 
[1, 1, 1, 2, 2, 3, 3], 
['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
[450, 300, 350, 500, 450, 370, 290].


In [1]:
import numpy as np
import pandas as pd

authors_dict = {
    "author_id" : [1, 2, 3],
    "author_name" : ['Тургенев', 'Чехов', 'Островский']
}

authors = pd.DataFrame(authors_dict)
print('Authors shape {}'.format(authors.shape))

book = pd.DataFrame({"author_id" : [1, 1, 1, 2, 2, 3, 3],
                     "book_title": ['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
                     "price" : [450, 300, 350, 500, 450, 370, 290]
                    })

print(f'Book index {book.index}')
print(f'Book columns {book.columns}')

Authors shape (3, 2)
Book index RangeIndex(start=0, stop=7, step=1)
Book columns Index(['author_id', 'book_title', 'price'], dtype='object')


### Задание 2
Получите датафрейм authors_price, соединив датафреймы authors и books по полю author_id.

In [4]:
author_price = pd.merge(authors, book, on='author_id', how='outer')

author_price

,author_id,author_name,book_title,price
0,1,Тургенев,Отцы и дети,450
1,1,Тургенев,Рудин,300
2,1,Тургенев,Дворянское гнездо,350
3,2,Чехов,Толстый и тонкий,500
4,2,Чехов,Дама с собачкой,450
5,3,Островский,Гроза,370
6,3,Островский,Таланты и поклонники,290


In [ ]:
### Задание 3
Создайте датафрейм top5, в котором содержатся строки из authors_price с пятью самыми дорогими книгами.

In [5]:
# вариант 1
top5 = author_price.sort_values(by='price', ascending=False, inplace=False)
top5.reset_index(inplace=True)
top5 = top5.loc[0:4,:]
print(f'Top 5, вариант 1:\n{top5}')
# вариант 2
top5 = author_price.nlargest(5, "price")
print(f'Top 5, вариант 2:\n{top5}')

Top 5, вариант 1:
   index  author_id author_name         book_title  price
0      3          2       Чехов   Толстый и тонкий    500
1      0          1    Тургенев        Отцы и дети    450
2      4          2       Чехов    Дама с собачкой    450
3      5          3  Островский              Гроза    370
4      2          1    Тургенев  Дворянское гнездо    350
Top 5, вариант 2:
   author_id author_name         book_title  price
3          2       Чехов   Толстый и тонкий    500
0          1    Тургенев        Отцы и дети    450
4          2       Чехов    Дама с собачкой    450
5          3  Островский              Гроза    370
2          1    Тургенев  Дворянское гнездо    350


### Задание 4
Создайте датафрейм authors_stat на основе информации из authors_price. В датафрейме authors_stat должны быть четыре столбца: author_name, min_price, max_price и mean_price,
в которых должны содержаться соответственно имя автора,минимальная, максимальная и средняя цена на книги этого автора.

In [6]:
author_price_tmp = author_price.copy()
# не понял как сделать так чтоб они добавились с группировку, так как иначе происходило просто замена
author_price_tmp["min_price"] = author_price_tmp["price"]
author_price_tmp["max_price"] = author_price_tmp["price"]
author_price_tmp["mean_price"] = author_price_tmp["price"]
authors_stat = author_price_tmp.groupby('author_name')
authors_stat.agg({"max_price" : "max", "min_price" : "min", "mean_price" : "mean"})

,max_price,min_price,mean_price
author_name,,,
Островский,370,290,330.000000
Тургенев,450,300,366.666667
Чехов,500,450,475.000000


### Задание 5**
Создайте новый столбец в датафрейме authors_price под названием cover, в нем будут располагаться данные о том, какая обложка у данной книги - твердая или мягкая. В этот столбец поместите данные из следующего списка:
['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая'].
Просмотрите документацию по функции pd.pivot_table с помощью вопросительного знака.Для каждого автора посчитайте суммарную стоимость книг в твердой и мягкой обложке. Используйте для этого функцию pd.pivot_table. При этом столбцы должны называться "твердая" и "мягкая", а индексами должны быть фамилии авторов. Пропущенные значения стоимостей заполните нулями, при необходимости загрузите библиотеку Numpy.
Назовите полученный датасет book_info и сохраните его в формат pickle под названием "book_info.pkl". Затем загрузите из этого файла датафрейм и назовите его book_info2. Удостоверьтесь, что датафреймы book_info и book_info2 идентичны.

In [13]:
author_price["cover"] = ['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая']
book_info = pd.pivot_table(author_price, values='price', index=['author_name'], columns=['cover'], aggfunc=np.sum).fillna(0)

book_info.to_pickle("book_info.pkl")
book_info = pd.read_pickle("book_info.pkl")

book_info

cover,мягкая,твердая
author_name,,
Островский,660.0,0.0
Тургенев,650.0,450.0
Чехов,0.0,950.0
